In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('chp4').getOrCreate()

In [0]:
csv_file = "/databricks-datasets/learning-spark-v2/flights/departuredelays.csv"
df = spark.read.csv(csv_file, header=True,inferSchema=True)
df.createOrReplaceTempView('us_flight_delay_tbl')

In [0]:
#find all flights whose distance is greater than 1,000 miles:
spark.sql("""select * from us_flight_delay_tbl where distance>1000 order by distance desc""").show(10)

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1090900| -3| 4330| JFK| HNL|
1050900| 98| 4330| JFK| HNL|
1080900| 14| 4330| JFK| HNL|
1020900| 1| 4330| JFK| HNL|
1040900| 111| 4330| JFK| HNL|
1060900| -2| 4330| JFK| HNL|
1070900| 3| 4330| JFK| HNL|
1010900| 6| 4330| JFK| HNL|
1110900| -4| 4330| JFK| HNL|
1030900| 784| 4330| JFK| HNL|
+-------+-----+--------+------+-----------+
only showing top 10 rows

In [0]:
#find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay:
spark.sql("""select * from us_flight_delay_tbl where origin='SFO' and destination ='ORD' and delay >=120 order by delay desc""").show(10)

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
2190925| 1638| 1604| SFO| ORD|
1031755| 396| 1604| SFO| ORD|
1022330| 326| 1604| SFO| ORD|
1051205| 320| 1604| SFO| ORD|
1190925| 297| 1604| SFO| ORD|
2171115| 296| 1604| SFO| ORD|
1071040| 279| 1604| SFO| ORD|
1051550| 274| 1604| SFO| ORD|
3120730| 266| 1604| SFO| ORD|
1261104| 258| 1604| SFO| ORD|
+-------+-----+--------+------+-----------+
only showing top 10 rows

In [0]:
spark.sql("""select date,delay,origin,destination, 
case 
when delay >360 then 'Very Long Delays'
when delay >=120 and delay <=360 then 'Long Delays'
when delay >=60 and delay <120 then 'Short Delay'
when delay > 0 and delay <60 then 'Tolerable Delay'
when delay =0 then 'No Delay'
else 'Early' end as Flight_Status
from us_flight_delay_tbl order by origin, delay desc""").show(10)

+-------+-----+------+-----------+-------------+
 date|delay|origin|destination|Flight_Status|
+-------+-----+------+-----------+-------------+
1220625| 333| ABE| ATL| Long Delays|
2120625| 305| ABE| ATL| Long Delays|
3021725| 275| ABE| ATL| Long Delays|
2150625| 257| ABE| ATL| Long Delays|
2211245| 247| ABE| DTW| Long Delays|
2211215| 247| ABE| ATL| Long Delays|
1220607| 219| ABE| ORD| Long Delays|
3201725| 211| ABE| ATL| Long Delays|
3121245| 197| ABE| DTW| Long Delays|
2141628| 192| ABE| ORD| Long Delays|
+-------+-----+------+-----------+-------------+
only showing top 10 rows

In [0]:
#find all flights whose distance is greater than 1,000 miles: DataFrame version
df.where(df.distance >1000).show(10)

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
1012355| 0| 1586| ABQ| JFK|
1022355| 158| 1586| ABQ| JFK|
1032355| 0| 1586| ABQ| JFK|
1042355| 0| 1586| ABQ| JFK|
1052355| 0| 1586| ABQ| JFK|
1062355| 0| 1586| ABQ| JFK|
1072359| 14| 1586| ABQ| JFK|
1082358| -4| 1586| ABQ| JFK|
1092358| 20| 1586| ABQ| JFK|
1102358| -2| 1586| ABQ| JFK|
+-------+-----+--------+------+-----------+
only showing top 10 rows

In [0]:
#find all flights between San Francisco (SFO) and Chicago (ORD) with at least a two-hour delay:
df.filter((df.origin =='SFO') & (df.destination == 'ORD') & (df.delay >120)).orderBy(df.delay.desc()).show(10)

+-------+-----+--------+------+-----------+
 date|delay|distance|origin|destination|
+-------+-----+--------+------+-----------+
2190925| 1638| 1604| SFO| ORD|
1031755| 396| 1604| SFO| ORD|
1022330| 326| 1604| SFO| ORD|
1051205| 320| 1604| SFO| ORD|
1190925| 297| 1604| SFO| ORD|
2171115| 296| 1604| SFO| ORD|
1071040| 279| 1604| SFO| ORD|
1051550| 274| 1604| SFO| ORD|
3120730| 266| 1604| SFO| ORD|
1261104| 258| 1604| SFO| ORD|
+-------+-----+--------+------+-----------+
only showing top 10 rows

In [0]:
from pyspark.sql.functions import when, col
df.select(col('*'),when(df.delay > 360,'Very Long Delay').when((df.delay>=120) & (df.delay<=360),'Long Delay')
         .when((df.delay>60) & (df.delay <120),'Tolerable Delay').when((df.delay>0) & (df.delay<60),'Short Delay')
         .when(df.delay == 0, 'On time').otherwise('Early').alias('Flight_Status')).show(10)

+-------+-----+--------+------+-----------+---------------+
 date|delay|distance|origin|destination| Flight_Status|
+-------+-----+--------+------+-----------+---------------+
1011245| 6| 602| ABE| ATL| Short Delay|
1020600| -8| 369| ABE| DTW| Early|
1021245| -2| 602| ABE| ATL| Early|
1020605| -4| 602| ABE| ATL| Early|
1031245| -4| 602| ABE| ATL| Early|
1030605| 0| 602| ABE| ATL| On time|
1041243| 10| 602| ABE| ATL| Short Delay|
1040605| 28| 602| ABE| ATL| Short Delay|
1051245| 88| 602| ABE| ATL|Tolerable Delay|
1050605| 9| 602| ABE| ATL| Short Delay|
+-------+-----+--------+------+-----------+---------------+
only showing top 10 rows